# Расчет параметров катушки из эксперимента

In [1]:
// подключим библиотеки
#r "nuget:XPlot.Plotly, 3.0.1"

using System.Net;
using System.Text;
using System.Globalization;
using XPlot.Plotly;
using System.Data;
using System.IO;
using System.Threading;
using System.Text.RegularExpressions;
using System.Linq;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package XPlot.Plotly version 3.0.1

In [2]:
// для расчета кривой градусы/метры оптимально взять 3-30 значений
var data = new double[,] 
{
    /*метры троса          */ {0, 10, 20, 30, 40, 50},
    /*градусы поворота вала*/ {0, 57295, 114591, 171887, 229183, 286478}
};

In [3]:
var plot = new Graph.Scatter() { x=Enumerable.Range(0, data.GetLength(1)).Select((x,i)=>data[0,i]), y=Enumerable.Range(0, data.GetLength(1)).Select((x,i)=>data[1,i])};
var chart = Chart.Plot(new [] {plot});
chart.WithYTitle("градусы поворота вала");
chart.WithXTitle("метры троса ");
display(chart);

In [4]:
// метод наименьших квадратов
public double[,] LeastSquareMethod(double[,] xyTable, int basis)
{
  var matrix = new double[basis, basis + 1];
  for (int i = 0; i < basis; i++)
  {
    for (int j = 0; j < basis; j++)
    {
      double sumA = 0, sumB = 0;
      for (int k = 0; k < xyTable.Length / 2; k++)
      {
        sumA += Math.Pow(xyTable[0, k], i) * Math.Pow(xyTable[0, k], j);
        sumB += xyTable[1, k] * Math.Pow(xyTable[0, k], i);
      }
      matrix[i, j] = sumA;
      matrix[i, basis] = sumB;
    }
  }
  return matrix;
}

In [5]:
//метод Гауса
public double[] Gauss(double[,] Matrix)
{
    int n = Matrix.GetLength(0); //Размерность начальной матрицы (строки)
    double[,] Matrix_Clone = new double[n, n + 1]; //Матрица-дублер
    for (int i = 0; i < n; i++)
        for (int j = 0; j < n + 1; j++)
            Matrix_Clone[i, j] = Matrix[i, j];

    //Прямой ход (Зануление нижнего левого угла)
    for (int k = 0; k < n; k++) //k-номер строки
    {
        for (int i = 0; i < n + 1; i++) //i-номер столбца
            Matrix_Clone[k, i] = Matrix_Clone[k, i] / Matrix[k, k]; //Деление k-строки на первый член !=0 для преобразования его в единицу
        for (int i = k + 1; i < n; i++) //i-номер следующей строки после k
        {
            double K = Matrix_Clone[i, k] / Matrix_Clone[k, k]; //Коэффициент
            for (int j = 0; j < n + 1; j++) //j-номер столбца следующей строки после k
                Matrix_Clone[i, j] = Matrix_Clone[i, j] - Matrix_Clone[k, j] * K; //Зануление элементов матрицы ниже первого члена, преобразованного в единицу
        }
        for (int i = 0; i < n; i++) //Обновление, внесение изменений в начальную матрицу
            for (int j = 0; j < n + 1; j++)
                Matrix[i, j] = Matrix_Clone[i, j];
    }

    //Обратный ход (Зануление верхнего правого угла)
    for (int k = n - 1; k > -1; k--) //k-номер строки
    {
        for (int i = n; i > -1; i--) //i-номер столбца
            Matrix_Clone[k, i] = Matrix_Clone[k, i] / Matrix[k, k];
        for (int i = k - 1; i > -1; i--) //i-номер следующей строки после k
        {
            double K = Matrix_Clone[i, k] / Matrix_Clone[k, k];
            for (int j = n; j > -1; j--) //j-номер столбца следующей строки после k
                Matrix_Clone[i, j] = Matrix_Clone[i, j] - Matrix_Clone[k, j] * K;
        }
    }

    //Отделяем от общей матрицы ответы
    double[] Answer = new double[n];
    for (int i = 0; i < n; i++)
        Answer[i] = Matrix_Clone[i, n];

    return Answer;
}

In [6]:
var lsm = LeastSquareMethod(data, 5);
var solution = Gauss(lsm);
display (solution);

index,value
0,-0.007936507929116488
1,5729.444576719577
2,0.006944444444437488
3,-9.259259259237385E-05
4,-2.136301999496989E-18


In [7]:
var outDict = new List<(double X, double Y)>();
for(var x=0D;x<50;x+=0.01)
{
    var y = solution.Select((a,i)=>Math.Pow(x,i)*a).Sum();
    outDict.Add((x,y));    
}
var outPlot = new Graph.Scatter() {x=outDict.Select(z=>z.X), y=outDict.Select(z=>z.Y)};
var outChart = Chart.Plot(new [] {outPlot});
outChart.WithYTitle("градусы поворота вала");
outChart.WithXTitle("метры троса");
display(outChart);

In [11]:
Console.WriteLine(string.Join(", ", solution.Select(x=>x.ToString(CultureInfo.InvariantCulture))));

-0.007936507929116488, 5729.444576719577, 0.006944444444437488, -9.259259259237385E-05, -2.136301999496989E-18
